In [1]:
# Creat By GF 2023-07-15

# Python3_Project_用户分层-RFM模型-2-计算数据.ipynb

In [11]:
import pandas as pd
#from datetime import *
from datetime import datetime

In [2]:
# 导入构建好的数据。

import Python3_Project_RFM_Building_Data

df = Python3_Project_RFM_Building_Data.df

In [13]:
df

,time,fruit,name,kilogram,price,region,amount
0,2019-01-01,香蕉,Tom,80,3.8,华南,304.0
1,2019-01-02,苹果,Tom,81,8.9,华北,720.9
2,2019-01-03,香蕉,Jhon,51,3.8,华南,193.8
3,2019-01-04,葡萄,Tom,96,12.8,西北,1228.8
4,2019-01-05,苹果,Michk,68,8.9,华北,605.2
...,...,...,...,...,...,...,...
1091,2021-12-27,梨,Xiaoming,83,5.8,华北,481.4
1092,2021-12-28,哈密瓜,Tom,99,15.8,西北,1564.2
1093,2021-12-29,橙子,Lym,75,6.8,华中,510.0
1094,2021-12-30,哈密瓜,Michk,88,15.8,西北,1390.4


In [14]:
# 用户分层 - RFM 模型。

In [15]:
# RFM 模型是衡量客户价值和创利能力的重要工具和手段。

# 通过这个模型能够反映一个用户的交期交易行为、交易的总体频率和总交易金额3项指标，通过3个指标来描述该客户的价值状况；同时依据这三项指标将客户划分为8类客户价值：

#     * Recency（R）是客户最近一次购买日期距离现在的天数，这个指标与分析的时间点有关，因此是变动的。理论上客户越是在近期发生购买行为，就越有可能复购

#     * Frequency（F）指的是客户发生购买行为的次数–最常购买的消费者，忠诚度也就较高。增加顾客购买的次数意味着能占有更多的时长份额。

#     * Monetary value（M）是客户购买花费的总金额。

In [16]:
# 通过 Pandas 的多个方法来分别求解这个 3 个指标，首先是 F 和 M : 每位客户的订单次数和总金额。

In [17]:
df_F_M = df.groupby("name").agg({"fruit":"count", "amount":"sum"}).reset_index().rename(columns={"fruit":"F", "amount":"M"})
df_F_M

,name,F,M
0,Jhon,161,102996.6
1,Jimmy,146,84559.0
2,Lym,159,92894.9
3,Michk,160,104454.8
4,Mike,136,90017.2
5,Tom,166,106195.8
6,Xiaoming,168,109398.7


In [18]:
# 通过 Pandas 求解 R 指标。

In [19]:
# 1. 先求解每个订单和当前时间的差值。

In [20]:
datetime.now() # -> 当前时间。

datetime.datetime(2023, 7, 18, 22, 49, 2, 795348)

In [21]:
df["R"] = df["time"].apply(lambda x: (datetime.now() - x).days)
df.head()

,time,fruit,name,kilogram,price,region,amount,R
0,2019-01-01,香蕉,Tom,80,3.8,华南,304.0,1659
1,2019-01-02,苹果,Tom,81,8.9,华北,720.9,1658
2,2019-01-03,香蕉,Jhon,51,3.8,华南,193.8,1657
3,2019-01-04,葡萄,Tom,96,12.8,西北,1228.8,1656
4,2019-01-05,苹果,Michk,68,8.9,华北,605.2,1655


In [22]:
# 2. 根据每个用户的这个差值 R 来进行升序排列，排在第一位的那条数据就是他最近购买记录：以 xiaoming 用户为例，最近一次是 2021 年 12 月 29 号，和当前时间的差值是 563 天。

In [23]:
df.sort_values(["name", "R"], ascending=[False, True]).head(10)

,time,fruit,name,kilogram,price,region,amount,R
1091,2021-12-27,梨,Xiaoming,83,5.8,华北,481.4,568
1082,2021-12-18,哈密瓜,Xiaoming,63,15.8,西北,995.4,577
1080,2021-12-16,香蕉,Xiaoming,91,3.8,华南,345.8,579
1072,2021-12-08,哈密瓜,Xiaoming,65,15.8,西北,1027.0,587
1066,2021-12-02,苹果,Xiaoming,59,8.9,华北,525.1,593
1052,2021-11-18,哈密瓜,Xiaoming,99,15.8,西北,1564.2,607
1043,2021-11-09,哈密瓜,Xiaoming,92,15.8,西北,1453.6,616
1032,2021-10-29,苹果,Xiaoming,66,8.9,华北,587.4,627
1015,2021-10-12,苹果,Xiaoming,80,8.9,华北,712.0,644
1008,2021-10-05,香蕉,Xiaoming,83,3.8,华南,315.4,651


In [24]:
# 3. 根据用户去重，保留第一条数据，这样便得到每个用户的 R 指标。

In [25]:
df_R = (df.sort_values(["name", "R"], ascending=[False, True]).drop_duplicates(["name"], keep="first")[["name", "R"]])
df_R

,name,R
1091,Xiaoming,568
1092,Tom,567
1095,Mike,564
1094,Michk,565
1093,Lym,566
1090,Jimmy,569
1056,Jhon,603


In [26]:
# 4. 数据合并得到 3 个指标。

In [27]:
df_RFM = pd.merge(df_F_M, df_R)
df_RFM

,name,F,M,R
0,Jhon,161,102996.6,603
1,Jimmy,146,84559.0,569
2,Lym,159,92894.9,566
3,Michk,160,104454.8,565
4,Mike,136,90017.2,564
5,Tom,166,106195.8,567
6,Xiaoming,168,109398.7,568


In [28]:
# Pandas 中的 .style 方法可以对 DataFrame 进行样式设置。
# .style 方法可以设置的项包括单元格的背景色、字体颜色、对齐方式等。
# 其中 cmap 参数指定了颜色映射表。
# 例如：
# .style.background_gradient(cmap='RdYlGn')
# 这里 cmap='RdYlGn' 使用了红黄绿三种颜色进行渐变。

df_RFM.style.background_gradient(cmap="Spectral_r") # -> cmap="Spectral_r" 指定了使用光谱色作为颜色映射。

,name,F,M,R
0,Jhon,161,102996.600000,603
1,Jimmy,146,84559.000000,569
2,Lym,159,92894.900000,566
3,Michk,160,104454.800000,565
4,Mike,136,90017.200000,564
5,Tom,166,106195.800000,567
6,Xiaoming,168,109398.700000,568


In [43]:
# 当数据量足够大，用户足够多的时候，就可以只用 RFM 模型来将用户分成 8 个类型。